# Replacing Anglegan Keras Loss Function

## AngleGAN
### https://github.com/adelC/3Dgan/blob/Anglegan/keras/AngleTrain3dGAN.py
    combined = Model(input=[latent],output=[fake, aux, ang, ecal, add_loss],name='combined_model')
    combined.compile(optimizer=RMSprop(lr),loss=['binary_crossentropy', 'mean_absolute_percentage_error', 'mae', 'mean_absolute_percentage_error', 'mean_absolute_percentage_error'],loss_weights=loss_weights)
    ...
    discriminator.evaluate( X, [y, aux_y, ang, ecal, add_loss], verbose=False, batch_size=batch_size)
        https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate
        Returns the loss value & metrics values for the model in test mode. Computation is done in batches (see the batch_size arg.)
    ...
    train_on_batch() is used to update weights
    ...
    https://github.com/adelC/3Dgan/blob/Anglegan/keras/AngleTrain3dGAN.py#L400
    disc_test_loss.append(discriminator.evaluate( X, [y, aux_y, ang, ecal, add_loss], verbose=False, batch_size=batch_size))

## Keras Source Code
### training.py --> def compile()
    defines the loss function, the optimizer and the metrics!
    self.compiled.loss = compile_utils.LossesContainer(metrics, weighted_metrics, output_names=self.output_names)
### LossesContainer() --> def build():
    maybe_broadcast_to_outputs(y_pred, self.losses)
        determines if losses/metrics should be applied to all outputs, applies losses to all y_pred --> returns nested structure of objects
    conform_to_outputs(y_pred, self_losses)
        maps struct to outputs structure
    nest.map_structure(self.get_loss_object,self._losses)
        apply function to each entry in structure --> returns new structure
    nest.flatten(self._losses)
        nested structure --> flat list
    self._create_metrics()
        creates per_output loss metrics (only for multi-output models)
    self._built()

# to replace keras compile

#https://keras.io/api/losses/
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

#Iterate over the batches of a dataset.
for x, y in dataset:
    with tf.GradientTape() as tape:
        # Forward pass.
        logits = model(x)
        # Loss value for this batch.
        loss_value = loss_fn(y, logits)
        # Add extra loss terms to the loss value.
        loss_value += sum(model.losses)

    # Update the weights of the model to minimize the loss value.
    gradients = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))

# main2 pgan compile

In [ ]:
for phase in range(1, num_phases + 1):
    optimizer_disc = tf.train.AdamOptimizer(learning_rate=d_lr, beta1=args.beta1, beta2=args.beta2)
    #...
    
SEE LOSS.PY

## Pgan Plan :)
    apply loss functions: loss_vector=[binarycrossentropy(fake), mae(ang), meanabsperc(ecal)]
    weight and sum losses: dot(loss_vector, weight_vector)
    how to integrate with pgan loss functions?!!

In [12]:
def physics_loss_term(optimizer, fake, ang, ecal, loss_weights):
    losses = np.array([bce(fake), mae(ang), mape(ecal)])   # calculate the losses and store in an array
    # make sure pgan weight vector is a np.array
    summed_loss = np.dot(loss_weights, losses)  # weight and sum the losses
    return summed_loss

## Keras Code for loss functions

In [4]:
#BINARY CROSS ENTROPY
#https://github.com/tensorflow/tensorflow/blob/r1.10/tensorflow/python/keras/backend.py
def binary_crossentropy(target, output, from_logits=False):
  """Binary crossentropy between an output tensor and a target tensor.
  Arguments:
      target: A tensor with the same shape as `output`.
      output: A tensor.
      from_logits: Whether `output` is expected to be a logits tensor.
          By default, we consider that `output`
          encodes a probability distribution.
  Returns:
      A tensor.
  """
  # Note: nn.sigmoid_cross_entropy_with_logits
  # expects logits, Keras expects probabilities.
  if not from_logits:
    # transform back to logits
    epsilon_ = _to_tensor(epsilon(), output.dtype.base_dtype)
    output = clip_ops.clip_by_value(output, epsilon_, 1 - epsilon_)
    output = math_ops.log(output / (1 - output))
  return nn.sigmoid_cross_entropy_with_logits(labels=target, logits=output)

from tensorflow.python.framework import smart_cond
#https://github.com/tensorflow/tensorflow/blob/v1.15.0/tensorflow/python/keras/losses.py#L976-L989
def binary_crossentropy(y_true, y_pred, from_logits=False, label_smoothing=0):  # pylint: disable=missing-docstring
  y_pred = ops.convert_to_tensor(y_pred)
  y_true = math_ops.cast(y_true, y_pred.dtype)
  label_smoothing = ops.convert_to_tensor(label_smoothing, dtype=K.floatx())

  def _smooth_labels():
    return y_true * (1.0 - label_smoothing) + 0.5 * label_smoothing

  y_true = smart_cond.smart_cond(label_smoothing,
                                 _smooth_labels, lambda: y_true)
  return K.mean(
      K.binary_crossentropy(y_true, y_pred, from_logits=from_logits), axis=-1)

In [ ]:
#MAE = MEAN ABSOLUTE ERROR
def mean_absolute_error(y_true, y_pred):
  y_pred = ops.convert_to_tensor(y_pred)
  y_true = math_ops.cast(y_true, y_pred.dtype)
  return K.mean(math_ops.abs(y_pred - y_true), axis=-1)

In [ ]:
#MEAN ABSOLUTE PERCENTAGE ERROR
#https://github.com/tensorflow/tensorflow/blob/v1.15.0/tensorflow/python/keras/losses.py#L285-L310
class MeanAbsolutePercentageError(LossFunctionWrapper):
  """Computes the mean absolute percentage error between `y_true` and `y_pred`.
  `loss = 100 * abs(y_true - y_pred) / y_true`
  Usage:
  ```python
  mape = tf.keras.losses.MeanAbsolutePercentageError()
  loss = mape([0., 0., 1., 1.], [1., 1., 1., 0.])
  print('Loss: ', loss.numpy())  # Loss: 5e+08
  ```
  Usage with the `compile` API:
  ```python
  model = tf.keras.Model(inputs, outputs)
  model.compile('sgd', loss=tf.keras.losses.MeanAbsolutePercentageError())
  ```
  """

  def __init__(self,
               reduction=losses_utils.ReductionV2.AUTO,
               name='mean_absolute_percentage_error'):
    super(MeanAbsolutePercentageError, self).__init__(
        mean_absolute_percentage_error, name=name, reduction=reduction)
#...

## Adapted loss functions -- in progress: apply sigmoid?, implement

In [2]:
import tensorflow as tf
import numpy as np

!['binary cross entropy'](1_rdBw0E-My8Gu3f_BOB6GMA.png "binary cross entropy")

In [16]:
# BINARY CROSS ENTROPY
def bce(output, target, from_logits=False):
    output = tf.convert_to_tensor(output)
    target = tf.convert_to_tensor(target)
    sigmoid_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=target, logits=output)
    loss= tf.reduce_mean(sigmoid_loss)
    return loss

output = np.array([0., 1., 2., 3.])
target = np.array([0.1, 1.2, 2.1, 3])
tf.print(bce(output, target))

-1.8045189423262791


!['MAE'](mean-absolute-error.png "MAE")

In [10]:
# MAE = MEAN ABSOLUTE ERROR
def mae(output, target):
    output = tf.convert_to_tensor(output)
    target = tf.convert_to_tensor(target)
    return tf.math.reduce_mean(tf.math.abs(target - output), axis=-1) / len(target)

output = np.array([0., 1., 2., 3.])
target = np.array([0.1, 1.2, 2.1, 3])
tf.print(mae(output, target))

0.025


!['MAPE'](dbf9cf7529-2.png "MAPE")

In [11]:
# MAPE = MEAN ABSOLUTE PERCENTAGE ERROR
def mape(output, target):
    target = tf.convert_to_tensor(target)
    output = tf.convert_to_tensor(output)
    output_without0 = tf.where(tf.equal(output, 0.), tf.ones_like(output), output)  # to avoid -inf losses when y_true is in denominator
    loss = (tf.math.reduce_sum(tf.math.abs(output-target)/output_without0) * 100) / len(target)
    return loss

output = np.array([0, 1.2, 10., 20., 32.])
target = np.array([0, 1., 11., 21., 30.])
tf.print(mape(output, target))

7.583333333333333


# Compile final loss (weighted and summed)

In [ ]:
# output=y_true, target=y_pred=plug model(x)
# I need to plug each value into array...what about conditional layers?
fake_loss = bce(fake_output, fake_target)
ang_loss = mae(ang_output, ang_target)
ecal_loss = mape(ecal_output, ecal_target)

In [ ]:
def sum_loss(fake_loss, ang_loss, ecal_loss, loss_weights):
    losses = np.array([fake_loss, ang_loss, ecal_loss])   # calculate the losses and store in an array
    loss_weights = loss_weights.numpy() # make sure pgan weight vector is a np.array
    summed_loss = np.dot(loss_weights, losses)  # weight and sum the losses
    return summed_loss

In [ ]:
def compile(optimizer, physics_loss):
    # Update the weights of the model to minimize the loss value.
    #gradients = tape.gradient(loss_value, model.trainable_weights)
    #optimizer.apply_gradients(zip(gradients, model.trainable_weights))